In [ ]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.common import LoggingConfig
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *

In [ ]:
credentials = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [ ]:
# Get the list of supported facilities
org_units = api.get('organisationUnitGroups/gH2DlwAo1ja', params={'fields':'organisationUnits'}).json()['organisationUnits']

In [ ]:
# period of analysis
start_period = '2024-04-01'
end_period = '2024-06-30'
period = '2024Q2'

In [15]:
delete_data = pd.DataFrame(columns=['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value'])
delete_data.to_csv('DELETE.csv', index=False)

elements = ['VcYITWFtRPl']

for org_unit in org_units:
    
    metadatas = []

    for element in elements:
        indicators_metadata = api.get('dataElementOperands', params={'paging':False, 'fields':'id,name', 'filter':f'dataElement.id:eq:{element}'})
        indicators_metadata = indicators_metadata.json()['dataElementOperands']

        for indicator in indicators_metadata:
            data = {'dataElement': indicator['id'].split('.')[0], 'period': '202406', 'orgUnit':org_unit['id'], 'categoryOptionCombo':indicator['id'].split('.')[1]}
            metadatas.append(data)

    metadatas = pd.json_normalize(metadatas)

    delete_data = pd.read_csv('DELETE.csv')
    delete_data = pd.concat([delete_data, metadatas])
    delete_data.to_csv('DELETE.csv', index=False, encoding='utf-8')